In [34]:
# TensorFlow и tf.keras
import tensorflow as tf
from tensorflow import keras

# NumPy библиотека
import numpy as np
print(tf.version)

import matplotlib.pyplot as plt

<module 'tensorflow._api.v2.version' from 'c:\\Users\\ramze\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\tensorflow\\_api\\v2\\version\\__init__.py'>


IMDB набор данных поставляется в комплекте с TensorFlow. Он уже предобработан так, что отзывы (последовательности слов) конвертированы в последовательности чисел, где каждое число представляет специфичное слово в словаре.

Следующий код загружает IMDB набор данных на вашу машину (или использует кэшированную копию если вы уже загружали его):

In [6]:
imdb = keras.datasets.imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

17464789/17464789 [==============================] - 4s 0us/step


Аргумент num_words=10000 сохраняет 10 000 наиболее часто встречающихся слов в тренировочных данных. Редкие слова отсеиваются, чтобы сохранить размер данных управляемым.

Рассмотрим формат данных. Набор приходит предобработанным: каждый пример - это массив чисел, представляющих слова в отзыве фильма. Каждая метка - число со значением 0 или 1, где 0 означает негативный отзыв, а 1 - позитивный отзыв.

In [9]:
"Training entries: {}, labels: {}".format(len(train_data), len(train_labels))

'Training entries: 25000, labels: 25000'

In [14]:
print(train_data[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


Отзывы фильмов могут быть разной длины. Код ниже показывает количество слов в первом и втором отзывах. Ввиду того, что вводные данные нейронной сети должны иметь одинаковую длину

In [15]:
len(train_data[0]), len(train_data[1])

(218, 189)

Преобразование чисел обратно в слова
Это может быть полезным - знать как преобразовывать числа обратно в текст. Здесь мы создадим вспомогательную функцию для выполнения запросов к объекту словаря, который содержит картирование чисел в строки:

In [16]:
# Индексы словаря связи слов и соотвествующих им чисел
word_index = imdb.get_word_index()

# Первые индексы зарезервированы
word_index = {k:(v+3) for k,v in word_index.items()}
word_index[""] = 0
word_index[""] = 1
word_index[""] = 2 # unknown
word_index[""] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
  return ' '.join([reverse_word_index.get(i, '?') for i in text])

1641221/1641221 [==============================] - 1s 0us/step


Теперь мы можем использовать decode_review функцию, чтобы отобразить текст для первого отзыва:

In [17]:
decode_review(train_data[0])

"? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you th

Подготовка данных
Отзывы - массивы чисел - должны быть преобразованы в тензоры прежде чем передавать их в нейронную сеть. Это преобразование может быть выполнено двумя путями:

 мы можем заполнить массивы, чтобы все они имели одинаковую длину, затем создадим цельночисловой тензор формы max_length * num_reviews (максимальная длина * количество отзывов). Мы можем использовать встроенный слой (embedding layer) способный обрабатывать такую форму как первый слой сети.

Ввиду того что отзывы должны быть одинаковой длины мы используем pad_sequences функцию, чтобы стандартизировать длину:

In [18]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data, value=word_index[""], padding='post', maxlen=256)

test_data = keras.preprocessing.sequence.pad_sequences(test_data, value=word_index[""], padding='post', maxlen=256)

Взглянем на длину примеров теперь:

In [19]:
len(train_data[0]), len(train_data[1])

(256, 256)

И проверим (теперь заполненный) первый отзыв:

In [20]:
print(train_data[0])

[   1   14   22   16   43  530  973 1622 1385   65  458 4468   66 3941
    4  173   36  256    5   25  100   43  838  112   50  670    2    9
   35  480  284    5  150    4  172  112  167    2  336  385   39    4
  172 4536 1111   17  546   38   13  447    4  192   50   16    6  147
 2025   19   14   22    4 1920 4613  469    4   22   71   87   12   16
   43  530   38   76   15   13 1247    4   22   17  515   17   12   16
  626   18    2    5   62  386   12    8  316    8  106    5    4 2223
 5244   16  480   66 3785   33    4  130   12   16   38  619    5   25
  124   51   36  135   48   25 1415   33    6   22   12  215   28   77
   52    5   14  407   16   82    2    8    4  107  117 5952   15  256
    4    2    7 3766    5  723   36   71   43  530  476   26  400  317
   46    7    4    2 1029   13  104   88    4  381   15  297   98   32
 2071   56   26  141    6  194 7486   18    4  226   22   21  134  476
   26  480    5  144   30 5535   18   51   36   28  224   92   25  104
    4 

In [21]:
# вводная форма - размер словаря, используемого для отзывов (10 000 слов)
vocab_size = 10000

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 16)          160000    
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

Создание валидационного набора
В ходе тренировки нам необходимо проверять аккуратность модели на данных, которые она еще не видела. Создадим валидационный набор, установив отдельно 10 000 примеров из изначально тренировочных данных. (Почему не использовать сразу тестовый набор? Наша цель - разработать и настроить нашу модель, используя только тренировочные данные, затем использовать тестовые данные только единожды, чтобы оценить аккуратность.)

In [24]:
x_val = train_data[:10000]
partial_x_train = train_data[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

In [30]:
%%time
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

Epoch 1/20
30/30 [==============================] - 3s 45ms/step - loss: 0.0933 - accuracy: 0.9753 - val_loss: 0.3121 - val_accuracy: 0.8830
Epoch 2/20
30/30 [==============================] - 1s 41ms/step - loss: 0.0885 - accuracy: 0.9774 - val_loss: 0.3167 - val_accuracy: 0.8819
Epoch 3/20
30/30 [==============================] - 1s 45ms/step - loss: 0.0841 - accuracy: 0.9797 - val_loss: 0.3213 - val_accuracy: 0.8811
Epoch 4/20
30/30 [==============================] - 1s 44ms/step - loss: 0.0803 - accuracy: 0.9799 - val_loss: 0.3269 - val_accuracy: 0.8814
Epoch 5/20
30/30 [==============================] - 1s 49ms/step - loss: 0.0769 - accuracy: 0.9808 - val_loss: 0.3333 - val_accuracy: 0.8792
Epoch 6/20
30/30 [==============================] - 1s 47ms/step - loss: 0.0741 - accuracy: 0.9809 - val_loss: 0.3393 - val_accuracy: 0.8788
Epoch 7/20
30/30 [==============================] - 1s 41ms/step - loss: 0.0704 - accuracy: 0.9830 - val_loss: 0.3434 - val_accuracy: 0.8797
Epoch 8/20
30

Оценка модели
Теперь посмотрим как модель справляется с работой. Два значения будут возвращены: потеря (количество, которое представляет нашу ошибку, чем оно ниже, тем лучше) и аккуратность.

In [31]:
results = model.evaluate(test_data, test_labels)

782/782 [==============================] - 2s 3ms/step - loss: 0.4594 - accuracy: 0.8599
